In [2]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.policy.expert_policy import ExpertPolicy
from IPython.display import Image
from pprint import pprint

In [3]:


traces = []
n_scenarios = 5
try:
    env=MetaDriveEnv(config={"map":"C",
                             "num_scenarios": n_scenarios,
                             "use_render": False}
    )

    for rep in range(n_scenarios):

        obs, step_info = env.reset(seed=3)
        step_info['rep'] = rep
        traces.append(step_info)
        print(f'{env.current_seed = }')
        while True:
            obs, reward, tm, tr, step_info = env.step([0,1])
            step_info['rep'] = rep
            traces.append(step_info)

            env.render(mode="topdown", 
                    window=False,
                    screen_record=True,
                    # scaling = 1,
                    # screen_size=(700, 870),
                    # camera_position=(60,-63)
                    )
        
            
            if tm or tr:
                break

        # env.top_down_renderer.generate_gif(f"{rep}.gif")

        # print(f'{obs = }')
        # pprint(step_info)

finally:
    env.close()

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 5


env.current_seed = 3


[INFO] Episode ended! Scenario Index: 3 Reason: out_of_road.


env.current_seed = 3


[INFO] Episode ended! Scenario Index: 3 Reason: out_of_road.


...

[INFO] Episode ended! Scenario Index: 3 Reason: out_of_road.


env.current_seed = 3


[INFO] Episode ended! Scenario Index: 3 Reason: out_of_road.


env.current_seed = 3


[INFO] Episode ended! Scenario Index: 3 Reason: out_of_road.


In [4]:
import pandas as pd

df = pd.DataFrame(traces)
df = df.select_dtypes('number')
grouped = df.groupby('rep')

for rep in range(1, n_scenarios):
    df_a = grouped.get_group(rep-1).reset_index(drop=True).drop('rep', axis=1)
    df_b = grouped.get_group(rep).reset_index(drop=True).drop('rep', axis=1)
    print(f'{df_a.equals(df_b) = }')



df_a.equals(df_b) = False
df_a.equals(df_b) = False
df_a.equals(df_b) = False
df_a.equals(df_b) = False


## Deterministic so far


# Using ADS


In [5]:
import os 


In [6]:

import torch

# torch.use_deterministic_algorithms(True)
# os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

from metadrive.examples.ppo_expert.torch_expert import torch_expert as expert
# from metadrive.examples.ppo_expert.numpy_expert import expert

# expert(deterministic=True)

In [7]:


traces = []
n_scenarios = 2
try:
    env=MetaDriveEnv(config={"map":"C",
                             "use_render": False,
                             "num_scenarios": n_scenarios}
    )

    for rep in range(n_scenarios):
        obs, step_info = env.reset(seed=0)
        step_info['obs'] = obs
        step_info['rep'] = rep

        traces.append(step_info)
        print(f'{env.current_seed = }')
        while True:
            action = expert(env.agent, deterministic=True)
            obs, reward, tm, tr, step_info = env.step(action)
            step_info['obs'] = obs
            step_info['rep'] = rep

            traces.append(step_info)

            env.render(mode="topdown", 
                    window=False,
                    screen_record=True,
                    )
        
            if tm or tr:
                break

        # env.top_down_renderer.generate_gif(f"ads_drive_{rep}.gif")

finally:
    env.close()

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 2
[INFO] Use Torch PPO expert.


env.current_seed = 0


[INFO] Episode ended! Scenario Index: 0 Reason: arrive_dest.


env.current_seed = 0


[INFO] Episode ended! Scenario Index: 0 Reason: arrive_dest.


In [8]:

df = pd.DataFrame(traces)
# df = df.select_dtypes('number')
grouped = df.groupby('rep')

def strip(df):
    return df.reset_index(drop=True).drop('rep', axis=1)

for rep in range(1, n_scenarios):
    df_a = strip(grouped.get_group(rep-1))
    df_b = strip(grouped.get_group(rep))
    print(f'{df_a.equals(df_b) = }')

df_a.equals(df_b) = False


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   overtake_vehicle_num  430 non-null    int64  
 1   velocity              430 non-null    float64
 2   steering              430 non-null    float64
 3   acceleration          430 non-null    float64
 4   step_energy           430 non-null    float64
 5   episode_energy        430 non-null    float64
 6   policy                430 non-null    object 
 7   crash_vehicle         430 non-null    bool   
 8   crash_object          430 non-null    bool   
 9   crash_building        430 non-null    bool   
 10  crash_human           430 non-null    bool   
 11  crash_sidewalk        430 non-null    bool   
 12  out_of_road           430 non-null    bool   
 13  arrive_dest           430 non-null    bool   
 14  max_step              430 non-null    bool   
 15  env_seed              4

In [10]:
def print_cols(a,b, step, column):
    v_a = a[column].iloc[step]
    v_b = b[column].iloc[step]
    print(f'At {step = } first run {column} was {v_a} and second {v_b}')

column = "velocity"


print_cols(df_a, df_b, 0, column)
print_cols(df_a, df_b, 1, column)


At step = 0 first run velocity was 0.0 and second 0.0
At step = 1 first run velocity was 0.27580300452669493 and second 0.2758030044881505


In [11]:
column = "action"
print_cols(df_a, df_b, 0, column)
print_cols(df_a, df_b, 1, column)
print_cols(df_a, df_b, 2, column)
print_cols(df_a, df_b, 3, column)


column = "velocity"
print_cols(df_a, df_b, 0, column)
print_cols(df_a, df_b, 1, column)
print_cols(df_a, df_b, 2, column)
print_cols(df_a, df_b, 3, column)

At step = 0 first run action was nan and second nan
At step = 1 first run action was [-0.12874566, 1.0] and second [-0.12874566, 1.0]
At step = 2 first run action was [-0.120055474, 1.0] and second [-0.120055474, 1.0]
At step = 3 first run action was [-0.11511787, 1.0] and second [-0.11511787, 1.0]
At step = 0 first run velocity was 0.0 and second 0.0
At step = 1 first run velocity was 0.27580300452669493 and second 0.2758030044881505
At step = 2 first run velocity was 0.5516061693681962 and second 0.551606169880776
At step = 3 first run velocity was 0.8273859744705098 and second 0.8273859746535176


In [12]:
df_a

,overtake_vehicle_num,velocity,steering,acceleration,step_energy,episode_energy,policy,crash_vehicle,crash_object,crash_building,...,max_step,env_seed,crash,step_reward,cost,obs,action,raw_action,episode_reward,episode_length
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,EnvInputPolicy,False,False,False,...,False,0,False,0.000000,0,"[0.097222224, 0.4861111, 0.5, 0.012345679, 0.5...",NaN,NaN,NaN,NaN
1,0,0.275803,-0.128746,1.000000,0.000217,0.000217,EnvInputPolicy,False,False,False,...,False,0,False,0.007856,0,"[0.09723745, 0.48609588, 0.5000851, 0.02460359...","[-0.12874566, 1.0]","(-0.12874566, 1.0)",0.007856,1.0
2,0,0.551606,-0.120055,1.000000,0.001097,0.001315,EnvInputPolicy,False,False,False,...,False,0,False,0.035550,0,"[0.09731457, 0.48601878, 0.5005525, 0.03686151...","[-0.120055474, 1.0]","(-0.120055474, 1.0)",0.043406,2.0
3,0,0.827386,-0.115118,1.000000,0.002032,0.003346,EnvInputPolicy,False,False,False,...,False,0,False,0.064335,0,"[0.09747127, 0.48586208, 0.50140524, 0.0491183...","[-0.11511787, 1.0]","(-0.11511787, 1.0)",0.107741,3.0
4,0,1.103185,-0.109745,1.000000,0.002984,0.006331,EnvInputPolicy,False,False,False,...,False,0,False,0.093113,0,"[0.0977118, 0.48562154, 0.5026712, 0.061376113...","[-0.10974529, 1.0]","(-0.10974529, 1.0)",0.200854,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,0,8.369781,-0.000060,-0.009972,0.036820,11.306304,EnvInputPolicy,False,False,False,...,False,0,False,0.875719,0,"[0.107680604, 0.47565272, 0.50905144, 0.384335...","[-5.966006e-05, -0.009972354]","(-5.966006e-05, -0.009972354)",238.279285,210.0
211,0,8.324006,0.002879,-0.028130,0.036661,11.342965,EnvInputPolicy,False,False,False,...,False,0,False,0.873263,0,"[0.10851265, 0.47482067, 0.50908834, 0.3823014...","[0.0028793407, -0.028129747]","(0.0028793407, -0.028129747)",239.152549,211.0
212,0,8.258084,0.005060,-0.040513,0.036358,11.379323,EnvInputPolicy,False,False,False,...,False,0,False,0.868054,0,"[0.10929024, 0.4740431, 0.5087924, 0.3793716, ...","[0.005060312, -0.040512912]","(0.005060312, -0.040512912)",240.020603,212.0
213,0,8.166093,0.006390,-0.056539,0.035926,11.415249,EnvInputPolicy,False,False,False,...,False,0,False,0.860508,0,"[0.109997034, 0.4733363, 0.5082514, 0.37528312...","[0.0063898903, -0.056538604]","(0.0063898903, -0.056538604)",240.881111,213.0


In [13]:
df_b

,overtake_vehicle_num,velocity,steering,acceleration,step_energy,episode_energy,policy,crash_vehicle,crash_object,crash_building,...,max_step,env_seed,crash,step_reward,cost,obs,action,raw_action,episode_reward,episode_length
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,EnvInputPolicy,False,False,False,...,False,0,False,0.000000,0,"[0.097222224, 0.4861111, 0.5, 0.012345679, 0.5...",NaN,NaN,NaN,NaN
1,0,0.275803,-0.128746,1.000000,0.000217,0.000217,EnvInputPolicy,False,False,False,...,False,0,False,0.007856,0,"[0.09723745, 0.48609588, 0.5000851, 0.02460359...","[-0.12874566, 1.0]","(-0.12874566, 1.0)",0.007856,1.0
2,0,0.551606,-0.120055,1.000000,0.001097,0.001315,EnvInputPolicy,False,False,False,...,False,0,False,0.035550,0,"[0.09731457, 0.48601878, 0.5005525, 0.03686151...","[-0.120055474, 1.0]","(-0.120055474, 1.0)",0.043406,2.0
3,0,0.827386,-0.115118,1.000000,0.002032,0.003346,EnvInputPolicy,False,False,False,...,False,0,False,0.064335,0,"[0.09747127, 0.48586208, 0.50140524, 0.0491183...","[-0.11511787, 1.0]","(-0.11511787, 1.0)",0.107741,3.0
4,0,1.103185,-0.109745,1.000000,0.002984,0.006331,EnvInputPolicy,False,False,False,...,False,0,False,0.093113,0,"[0.0977118, 0.48562154, 0.5026712, 0.061376113...","[-0.10974529, 1.0]","(-0.10974529, 1.0)",0.200854,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,0,8.369767,-0.000060,-0.009959,0.036820,11.306302,EnvInputPolicy,False,False,False,...,False,0,False,0.875719,0,"[0.10768055, 0.47565278, 0.5090511, 0.3843353,...","[-6.02114e-05, -0.009959301]","(-6.02114e-05, -0.009959301)",238.279231,210.0
211,0,8.324006,0.002879,-0.028121,0.036661,11.342963,EnvInputPolicy,False,False,False,...,False,0,False,0.873263,0,"[0.108512595, 0.47482073, 0.5090879, 0.3823014...","[0.00287867, -0.028120926]","(0.00287867, -0.028120926)",239.152494,211.0
212,0,8.258089,0.005060,-0.040509,0.036358,11.379321,EnvInputPolicy,False,False,False,...,False,0,False,0.868054,0,"[0.10929019, 0.47404313, 0.50879186, 0.3793718...","[0.0050596413, -0.040509276]","(0.0050596413, -0.040509276)",240.020548,212.0
213,0,8.166097,0.006389,-0.056539,0.035926,11.415247,EnvInputPolicy,False,False,False,...,False,0,False,0.860508,0,"[0.10999698, 0.47333637, 0.508251, 0.37528333,...","[0.0063891453, -0.056538843]","(0.0063891453, -0.056538843)",240.881056,213.0
